# Overview
This notebook contrasts the average running time when applying a threshold function to an array in parallel or sequentially. This serves to answer the following questions: should the threshold function be applied in parallel or sequentially to the nodes of small Fuzzy Cognitive Maps?

# Experimental set-up

In [1]:
from math import exp
import multiprocessing as mp
import time
from copy import deepcopy
import numpy as np

In [2]:
def transferFunction(element):
    return (1/(1+exp(-element)))

def mean(L):
    return np.array(L).mean()

In [3]:
parTimeAve = [0]*501 #average time spent when doing it in parallel
serialTimeAve = [0]*501 #average time spent when doing it in serial
parVar=[0]*501#standard deviation
serialVar=[0]*501

In [4]:
#given a number of factors n (e.g., 4), compute the time it takes to apply a threshold
#function to an array of size n, averaged over 500 repeats.
def runExperiments(n):
    parTime = []
    serialTime = []
    pool = mp.Pool()
    for i in range(0,500):
        ex = np.random.random_sample(n,)#generates one random instance of n numbers
        #parallel benchmark
        sTime = time.time()
        results = pool.map_async(transferFunction, ex)
        #results = [pool.apply_async(transferFunction, (element,)) for element in ex]
        #when ALL parallel processes are done with the array, THEN we take the time
        #it's the time for the task to be fully done.
        parTime.append(time.time() - sTime)
        
        #sequential benchmark
        sTime = time.time()
        results = map(transferFunction, ex)
        serialTime.append(time.time()-sTime)
        
    parTimeAve[n]=mean(parTime)
    parVar[n]=np.var(parTime)
    serialTimeAve[n]=mean(serialTime)
    serialVar[n]=np.var(serialTime)

# Generating results
For all values from 5 to 500 by steps of 5 (so 5, 10, 15, 20, etc.) we call the function above to get the average time for sequential versus parallel processing. These results are written to a file, which can be analyzed or plotted externally.

In [5]:
#generates all results from 5 to 500 by steps of 5
def generateResults():
    current=0
    f=open("Results.csv",'w')
    f.write("n,Sequential (mean), Sequential (std), Parallel (mean), Parallel (std)\n")
    while current<500:
        runExperiments(current)
        f.write(str(current)+","+str(serialTimeAve[current])+","+str(serialVar[current])+","+str(parTimeAve[current])+","+str(parVar[current])+"\n")
        current+=5
        print ".",
        if current%100==0:
            print "\n"
    f.close()

In [6]:
generateResults()

. . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . 

